# Exemples d'implémentation d'algorithmes avec dask

## Nombres premiers

Voici un exemple d'implémentation d'un algorithme pour trouver les nombres premiers jusqu'à un certain nombre `n` en utilisant Dask.

In [5]:
from dask.distributed import Client
client = Client()

In [ ]:
client.close()

In [ ]:
client.dashboard_link

In [40]:
import time

def is_prime(x):
    for i in range(2, x // 2):
      if x % i == 0:
        return None
    return x

def launch(n):
  print(f"running for {n}")
  input = range(3, n)
  start = time.perf_counter()
  results = [x for x in input if is_prime(x) != None]
  end = time.perf_counter()
  print(f"Single threaded time: {end - start:.6f} seconds")

  start = time.perf_counter()
  futures = client.map(is_prime, input)
  results = client.gather(futures)
  end = time.perf_counter()
  print(f"Dask time: {end - start:.6f} seconds")

launch(100)
launch(10000)

running for 100
Single threaded time: 0.000031 seconds
Dask time: 0.059348 seconds
running for 10000
Single threaded time: 0.099690 seconds
Dask time: 5.287371 seconds


Encodage César de niveau 1 qui incrémente chaque lettre de l'alphabet d'une position. Par exemple, "abc" deviendrait "bcd". Faire aussi le décodage inverse.

In [19]:
def encode(c):
    if c == "z":
       return "a"
    return chr(ord(c) + 1)

def decode(c):
    if c == "a":
       return "z"
    return chr(ord(c) - 1)

input = "azhello"
futures = client.map(encode, input)
encoded = client.gather(futures)

print(encoded, "".join(encoded))


futures = client.map(decode, encoded)
output = client.gather(futures)

print("".join(output))

print("input equal to decoded ?", input == "".join(output))


['b', 'a', 'i', 'f', 'm', 'm', 'p'] baifmmp
azhello
input equal to decoded ? True


Faire la partie 1 du day [4 de advent of code 2025](https://adventofcode.com/2025/day/4). Trouver les cases qui ont moins de 4 `@` dans les cases adjacentes.

In [39]:
input = """
..@@.@@@@.
@@@.@.@.@@
@@@@@.@.@@
@.@@@@..@.
@@.@@@@.@@
.@@@@@@@.@
.@.@.@.@@@
@.@@@.@@@@
.@@@@@@@@.
@.@.@@@.@.
"""

lines = input.split()
print("lines", lines)
line_count = len(lines)
col_count = len(lines[0])

def count_rolls(index):
    line = index[0]
    col = index[1]
    if lines[line][col] != "@":
        return (index, 10)
    count = 0
    for i in range(line - 1, line + 2):
        for j in range(col - 1, col + 2):
            if (i, j) != (line, col) and 0 <= i < line_count and 0 <= j < col_count and lines[i][j] == "@":
                count += 1
    return (index, count)

# Produit cartésien
indices = [(x, y) for x in range(line_count) for y in range(col_count)]
print(indices)

futures = client.map(count_rolls, indices)
output = client.gather(futures)
print(output)
result = [x for x in output if x[1] < 4]
print(len(result), result)

lines ['..@@.@@@@.', '@@@.@.@.@@', '@@@@@.@.@@', '@.@@@@..@.', '@@.@@@@.@@', '.@@@@@@@.@', '.@.@.@.@@@', '@.@@@.@@@@', '.@@@@@@@@.', '@.@.@@@.@.']
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7), (7, 8), (7, 9), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (8, 9), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9)]
[((0, 0), 10), ((0, 1), 10), ((0, 2), 3), ((0, 3), 3